In [1]:
import sys
import os

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp
import chex
import time
#disable jax JIT
#jax.config.update("jax_disable_jit", True)

from tqdm.autonotebook import trange, tqdm
from functools import partial
from JAxtar.bgpq import HashTableIdx_HeapValue, BGPQ

/tmp/ipykernel_13247/3397891388.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange, tqdm


In [3]:
max_size = int(1e7)
batch_size = 8192
filled_sample = 8192

In [4]:
#cpu heap test
import heapq as hq
import numpy as np
heap = []
for i in trange(max_size):
    hq.heappush(heap, np.random.rand())

for i in trange(max_size):
    out = hq.heappop(heap)

  0%|          | 0/10000000 [00:00<?, ?it/s]

  0%|          | 0/10000000 [00:00<?, ?it/s]

In [5]:
heap = BGPQ.build(int(1e7), batch_size, HashTableIdx_HeapValue)

In [6]:
heap_key = jax.random.uniform(jax.random.PRNGKey(0), shape=(filled_sample,), minval=0, maxval=10)
value = jax.vmap(HashTableIdx_HeapValue.default)(jnp.arange(filled_sample))
make_batch = BGPQ.make_batched(heap_key, value, batch_size)

In [7]:
pbar = tqdm(total=int(max_size))
for i in range(0, max_size, filled_sample):
    heap_key = jax.random.uniform(jax.random.PRNGKey(i), shape=(filled_sample,), minval=0, maxval=10)
    value = jax.vmap(HashTableIdx_HeapValue.default)(jnp.arange(filled_sample))
    heap = BGPQ.insert(heap, heap_key, value)
    pbar.update(filled_sample)
pbar.close()

  0%|          | 0/10000000 [00:00<?, ?it/s]

In [8]:
pbar = tqdm(total=int(heap.size))
last_min = jnp.inf
last_max = -jnp.inf
print(f"heap.size: {heap.size}")
while heap.size > 0:
    heap, min_key, min_val = BGPQ.delete_mins(heap)
    pbar.update(batch_size)
    minimum = jnp.min(min_key)
    maximum = jnp.max(min_key)
    if minimum < last_max:
        print("Error last_max", last_max, "minimum", minimum)
        print("Not sorted")
        break
    last_min = minimum
    last_max = maximum
    pbar.set_description_str(f"heap.size: {heap.size}, min_key: {minimum:.2f}, max_key: {maximum:.2f}")
pbar.close()

  0%|          | 0/10002432 [00:00<?, ?it/s]

heap.size: 10002432


In [9]:
print(heap.key_buffer)

[inf inf inf ... inf inf inf]
